In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Re-initialize video capture
cap = cv2.VideoCapture(video_path)
ret, first_frame = cap.read()
cap.release()

# Initialize parameters for heatmap video
heatmap_vid_path = 'heatmap_tracking.mp4'
heatmap_fps = fps  # Match original video FPS
heatmap_fourcc = cv2.VideoWriter_fourcc(*'mp4v')
heatmap_writer = cv2.VideoWriter(heatmap_vid_path, heatmap_fourcc, heatmap_fps, (width, height))

# Re-process video with heatmap accumulation
cap = cv2.VideoCapture(video_path)
tracker = cv2.TrackerCSRT_create()
tracker.init(first_frame, init_bbox)

# Initialize heatmap matrix
heatmap = np.zeros((height, width), dtype=np.float32)
kernel_size = (25, 25)  # Gaussian blur kernel size

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    success, bbox = tracker.update(frame)
    
    if success:
        # Update tracking visualization
        x, y, w, h = [int(v) for v in bbox]
        current_center = (x + w//2, y + h//2)
        
        # Update heatmap (add current position)
        if 0 <= current_center[0] < width and 0 <= current_center[1] < height:
            heatmap[current_center[1], current_center[0]] += 1
        
        # Smooth the heatmap
        smoothed_heatmap = cv2.GaussianBlur(heatmap, kernel_size, 0)
        
        # Normalize and colorize
        norm_heatmap = cv2.normalize(smoothed_heatmap, None, 0, 255, cv2.NORM_MINMAX)
        color_heatmap = cv2.applyColorMap(norm_heatmap.astype(np.uint8), cv2.COLORMAP_JET)
        
        # Blend heatmap with frame
        overlay = cv2.addWeighted(frame, 0.7, color_heatmap, 0.3, 0)
        
        # Draw tracking elements
        cv2.rectangle(overlay, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(overlay, f"Frame: {frame_idx}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    else:
        overlay = frame.copy()
        cv2.putText(overlay, "Tracking Lost", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    heatmap_writer.write(overlay)
    frame_idx += 1
    print(f"Heatmap video progress: {frame_idx}/{total_frames}")

cap.release()
heatmap_writer.release()

print("Processing complete. Output files created:")
print(f"- Tracked video: {output_path}")
print(f"- Heatmap video: {heatmap_vid_path}")
print("- Static heatmap image: tracking_heatmap.png")

Heatmap video progress: 1/500
Heatmap video progress: 2/500
Heatmap video progress: 3/500
Heatmap video progress: 4/500
Heatmap video progress: 5/500
Heatmap video progress: 6/500
Heatmap video progress: 7/500
Heatmap video progress: 8/500


Heatmap video progress: 9/500
Heatmap video progress: 10/500
Heatmap video progress: 11/500
Heatmap video progress: 12/500
Heatmap video progress: 13/500
Heatmap video progress: 14/500
Heatmap video progress: 15/500
Heatmap video progress: 16/500
Heatmap video progress: 17/500
Heatmap video progress: 18/500
Heatmap video progress: 19/500
Heatmap video progress: 20/500
Heatmap video progress: 21/500
Heatmap video progress: 22/500
Heatmap video progress: 23/500
Heatmap video progress: 24/500
Heatmap video progress: 25/500
Heatmap video progress: 26/500
Heatmap video progress: 27/500
Heatmap video progress: 28/500
Heatmap video progress: 29/500
Heatmap video progress: 30/500
Heatmap video progress: 31/500
Heatmap video progress: 32/500
Heatmap video progress: 33/500
Heatmap video progress: 34/500
Heatmap video progress: 35/500
Heatmap video progress: 36/500
Heatmap video progress: 37/500
Heatmap video progress: 38/500
Heatmap video progress: 39/500
Heatmap video progress: 40/500
Heatmap v

In [8]:
from collections import deque
import cv2
import numpy as np

# Re-initialize video capture
cap = cv2.VideoCapture(video_path)
ret, first_frame = cap.read()
cap.release()

# Initialize parameters for heatmap video
heatmap_vid_path = 'dynamic_heatmap.mp4'
heatmap_fps = fps
heatmap_fourcc = cv2.VideoWriter_fourcc(*'mp4v')
heatmap_writer = cv2.VideoWriter(heatmap_vid_path, heatmap_fourcc, heatmap_fps, (width, height))

# Re-process video with dynamic heatmap
cap = cv2.VideoCapture(video_path)
tracker = cv2.TrackerCSRT_create()
tracker.init(first_frame, init_bbox)

# Heatmap configuration
kernel_size = (25, 25)
history_duration = 0.5  # seconds
history_frames = int(fps * history_duration)
heatmap_history = deque(maxlen=history_frames*2)  # Safety buffer

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    success, bbox = tracker.update(frame)
    
    current_heatmap = np.zeros((height, width), dtype=np.float32)
    
    if success:
        # Update tracking and history
        x, y, w, h = [int(v) for v in bbox]
        current_center = (x + w//2, y + h//2)
        
        # Add current position to history with frame index
        heatmap_history.append((frame_idx, current_center))
        
        # Remove old entries (older than 2 seconds)
        cutoff = frame_idx - history_frames
        while heatmap_history and heatmap_history[0][0] < cutoff:
            heatmap_history.popleft()
        
        # Accumulate positions in current window
        for timestamp, (cx, cy) in heatmap_history:
            if 0 <= cx < width and 0 <= cy < height:
                current_heatmap[cy, cx] += 1
        
        # Apply Gaussian blur
        smoothed_heatmap = cv2.GaussianBlur(current_heatmap, kernel_size, 0)
        
        # Normalize and colorize
        norm_heatmap = cv2.normalize(smoothed_heatmap, None, 0, 255, cv2.NORM_MINMAX)
        color_heatmap = cv2.applyColorMap(norm_heatmap.astype(np.uint8), cv2.COLORMAP_JET)
        
        # Blend heatmap with frame
        overlay = cv2.addWeighted(frame, 0.7, color_heatmap, 0.3, 0)
        
        # Draw tracking elements
        cv2.rectangle(overlay, (x, y), (x+w, y+h), (0, 255, 0), 2)
    else:
        overlay = frame.copy()
        cv2.putText(overlay, "Tracking Lost", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    # Add frame counter
    cv2.putText(overlay, f"Time: {frame_idx/fps:.1f}s", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    heatmap_writer.write(overlay)
    frame_idx += 1
    print(f"Processing frame {frame_idx} - History entries: {len(heatmap_history)}")

cap.release()
heatmap_writer.release()

print("Processing complete. Files created:")
print(f"- Dynamic heatmap video: {heatmap_vid_path}")
print(f"- Original tracking video: {output_path}")
print("- Static heatmap image: tracking_heatmap.png")

Processing frame 1 - History entries: 1
Processing frame 2 - History entries: 2
Processing frame 3 - History entries: 3
Processing frame 4 - History entries: 4
Processing frame 5 - History entries: 5
Processing frame 6 - History entries: 6
Processing frame 7 - History entries: 7
Processing frame 8 - History entries: 8
Processing frame 9 - History entries: 9


Processing frame 10 - History entries: 10
Processing frame 11 - History entries: 11
Processing frame 12 - History entries: 12
Processing frame 13 - History entries: 13
Processing frame 14 - History entries: 13
Processing frame 15 - History entries: 13
Processing frame 16 - History entries: 13
Processing frame 17 - History entries: 13
Processing frame 18 - History entries: 13
Processing frame 19 - History entries: 13
Processing frame 20 - History entries: 13
Processing frame 21 - History entries: 13
Processing frame 22 - History entries: 13
Processing frame 23 - History entries: 13
Processing frame 24 - History entries: 13
Processing frame 25 - History entries: 13
Processing frame 26 - History entries: 13
Processing frame 27 - History entries: 13
Processing frame 28 - History entries: 13
Processing frame 29 - History entries: 13
Processing frame 30 - History entries: 13
Processing frame 31 - History entries: 13
Processing frame 32 - History entries: 13
Processing frame 33 - History entr

In [3]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Set video path and initialize capture
video_path = '/home/ubuntu/project_dl/v_ITo3sCnpw_k_c012.mp4'
cap = cv2.VideoCapture(video_path)

# Read first frame and manually set bounding box
ret, first_frame = cap.read()
if not ret:
    raise ValueError("Failed to read video")

# Manually define initial bounding box (x, y, width, height) based on ground truth for id 8 in the first frame
init_bbox = (231, 234, 41, 70)  # From gt.txt line 9 for frame 1, id 8: 1, 8, 231, 234, 41, 70, ...
# Initialize CSRT tracker
tracker = cv2.TrackerCSRT_create()
tracker.init(first_frame, init_bbox)

# Get video properties for output
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Prepare output video writer
output_path = 'tracked_output_v_ITo3sCnpw_k_c012.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Tracked bounding boxes and centers
tracked_bboxes = []
centers = []

# Process video frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    success, bbox = tracker.update(frame)

    if success:
        # Extract bounding box coordinates
        x, y, w, h = [int(v) for v in bbox]
        tracked_bbox = (x, y, w, h)
        tracked_bboxes.append(tracked_bbox)
        current_center = (x + w // 2, y + h // 2)
        centers.append(current_center)

        # Draw bounding box and path on frame for visualization (optional)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        if len(centers) > 1:
            cv2.polylines(frame, [np.array(centers, dtype=np.int32)], False, (0, 255, 0), 2)
    else:
        tracked_bboxes.append(None) # Store None if tracking fails
        cv2.putText(frame, "Tracking Lost", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out.write(frame)
    frame_count += 1
    print(f"Processed frame {frame_count}/{total_frames}")

# Release resources
cap.release()
out.release()

print("Tracking finished. Output video saved to:", output_path)

Processed frame 1/500
Processed frame 2/500
Processed frame 3/500
Processed frame 4/500
Processed frame 5/500
Processed frame 6/500
Processed frame 7/500
Processed frame 8/500
Processed frame 9/500
Processed frame 10/500
Processed frame 11/500
Processed frame 12/500
Processed frame 13/500
Processed frame 14/500
Processed frame 15/500
Processed frame 16/500
Processed frame 17/500
Processed frame 18/500
Processed frame 19/500
Processed frame 20/500
Processed frame 21/500
Processed frame 22/500
Processed frame 23/500
Processed frame 24/500
Processed frame 25/500
Processed frame 26/500
Processed frame 27/500
Processed frame 28/500
Processed frame 29/500
Processed frame 30/500
Processed frame 31/500
Processed frame 32/500
Processed frame 33/500
Processed frame 34/500
Processed frame 35/500
Processed frame 36/500
Processed frame 37/500
Processed frame 38/500
Processed frame 39/500
Processed frame 40/500
Processed frame 41/500
Processed frame 42/500
Processed frame 43/500
Processed frame 44/5

Processed frame 48/500
Processed frame 49/500
Processed frame 50/500
Processed frame 51/500
Processed frame 52/500
Processed frame 53/500
Processed frame 54/500
Processed frame 55/500
Processed frame 56/500
Processed frame 57/500
Processed frame 58/500
Processed frame 59/500
Processed frame 60/500
Processed frame 61/500
Processed frame 62/500
Processed frame 63/500
Processed frame 64/500
Processed frame 65/500
Processed frame 66/500
Processed frame 67/500
Processed frame 68/500
Processed frame 69/500
Processed frame 70/500
Processed frame 71/500
Processed frame 72/500
Processed frame 73/500
Processed frame 74/500
Processed frame 75/500
Processed frame 76/500
Processed frame 77/500
Processed frame 78/500
Processed frame 79/500
Processed frame 80/500
Processed frame 81/500
Processed frame 82/500
Processed frame 83/500
Processed frame 84/500
Processed frame 85/500
Processed frame 86/500
Processed frame 87/500
Processed frame 88/500
Processed frame 89/500
Processed frame 90/500
Processed f